# Import librairies et data #

In [ ]:
!pip install ta
!pip install investpy
!pip install pandas-ta

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

import investpy
import pandas_ta as ta

from copy import copy
import matplotlib.pyplot as plt
import plotly.figure_factory as ff
import plotly.express as px
import plotly.graph_objects as go

import math
import statistics as stats
import random
import multiprocessing as mp

In [ ]:
df_source = investpy.get_etf_historical_data(etf='Lyxor UCITS NASDAQ-100 Daily Leverage',
                                            country='France',
                                            from_date='01/10/2006',
                                            to_date='31/12/2011')

df_source.drop(columns=['Currency','Exchange'], inplace=True)
#df_source.reset_index(inplace=True)

In [ ]:
df_source.isnull().sum()

Open      0
High      0
Low       0
Close     0
Volume    0
dtype: int64

In [ ]:
df_source.tail(1)

,Open,High,Low,Close,Volume
Date,,,,,
2011-12-30,24.07,24.07,23.9,23.94,32433


In [ ]:
df_source_allTime = investpy.get_etf_historical_data(etf='Lyxor UCITS NASDAQ-100 Daily Leverage',
                                            country='France',
                                            from_date='01/10/2006',
                                            to_date='14/04/2022')

df_source_allTime.drop(columns=['Currency','Exchange'], inplace=True)
df_source.isnull().sum()

Open      0
High      0
Low       0
Close     0
Volume    0
dtype: int64

In [ ]:
df_source_allTime.tail(1)

,Open,High,Low,Close,Volume
Date,,,,,
2022-04-14,724.3,726.8,708.0,711.3,3226


In [ ]:
bh1 = ((df_source['Close'].iloc[-1] - df_source['Close'].iloc[0])/df_source['Close'].iloc[0])*100
bh2 = ((df_source_allTime['Close'].iloc[-1] - df_source_allTime['Close'].iloc[0])/df_source_allTime['Close'].iloc[0])*100
print("Buy & Hold 2006-2011 : {:.2f} %".format(bh1))
print("Buy & Hold 2006-2022 : {:.2f} %".format(bh2))

Buy & Hold 2006-2011 : 37.27 %
Buy & Hold 2006-2022 : 3978.56 %


# Fonctions #

In [ ]:
def generate_ta(df_in,atr,multip,vwma_l):
  ''' Genère un Dataframe contenant Supertrend + VWMA '''

  df = df_in.ta.supertrend(high=df_in["High"], low=df_in["Low"], close=df_in["Close"], length = atr, multiplier = multip)

  # Nom des colonnes pour saisir la 1ere variabilisée
  col_list = df.columns
  # Suppresion des lignes vides, Subset sur la colonne supertrend
  df.dropna(subset=[col_list[0]], inplace=True)

  # Suppression de la 1ère ligne dans la valeur supertrend est = 0
  s_supertrend = df.iloc[1:,0]

  df_st = df_in.join(s_supertrend)

  # Suppression des lignes sans correspondance entre date et une valeur supertrend
  col_list = df_st.columns
  # Colonne supertrend étant la dernière
  supertrend_col = col_list[-1]
  # Garder uniquement date, close et supertrend
  df_st = df_st[["Close",supertrend_col]]
  # Suppression des lignes sans indicateur
  df_st.dropna(subset=[supertrend_col], inplace=True)

  # Ajout colonne VWMA
  df_st["vwma_"+str(vwma_l)] = round( df_in.ta.vwma(close=df_in["Close"], volume=df_in["Volume"], length=vwma_l) ,2)

  # Idem : suppression des lignes sans correspondace entre date et vwma
  df_st.dropna(subset=["vwma_"+str(vwma_l)], inplace=True)
  
  # Retour des dates en colonne plutôt qu'index
  df_st.reset_index(inplace=True)

  df_st.columns=['Date','Close','Supertrend','VWMA']
  
  return df_st

In [ ]:
def find_entries(df):
  ''' Obtenir les index des dates auxquelles la cloture est supérieure au Supertrend et la VWMA 55
  Croisement vers le haut de l'une ou de l'autre '''

  # Ajout des colonnes lag
  shift_vwma = df[df.columns[-1]].shift(1)
  shift_st = df[df.columns[-2]].shift(1)
  shift_close = df["Close"].shift(1)

  shift_vwma[0]=shift_vwma[1]
  df['VWMA_lag']=shift_vwma

  shift_st[0]=shift_st[1]
  df['ST_lag']=shift_st

  shift_close[0]=shift_close[1]
  df['Close_lag']=shift_close

  e_l = df.index[ 
                  ( (df["Close"] > df["Supertrend"]) 
                  & (df["Close_lag"] <= df["ST_lag"]) 
                  & (df["Close"] >= df["VWMA"]) )
                  |  
                  ( (df["Close"] > df["VWMA"]) 
                  & (df["Close_lag"] <= df["VWMA_lag"]) 
                  & (df["Close"] >= df["Supertrend"]) )
                ].to_list()

  return e_l

In [ ]:
def fix_oob_index(a, limite):
  ''' Necessaire pour corriger si l'entrée est trop proche de notre date limite max.
  Doit préceder generate_future_dates en conséquence'''
  for element in range(0,len(a)):
    if a[element] > limite :
      a[element] = limite
  return a

In [ ]:
def generate_future_dates(df, entries_list):
  '''Générer les dates futures après identification des entrées
  Puis calcul des variations'''

  arr_entries = np.asarray(entries_list)

  arr_e2s = arr_entries + 10
  arr_e1m = arr_entries + 21
  
  # Aggregation des matrices sous forme de liste en vue de traitements en boucle
  dates_ech = [arr_e2s, arr_e1m]

  # On fixe le plus grand index possible, au cas où la projection 5, 10j ou 1 mois en avant soit out of bounds
  date_limite = df.shape[0]-1

  # Correction des éventuelles dates futures out of bounds, selon fonction définie précedemment
  for echeance in dates_ech:
    echeance = fix_oob_index(echeance,date_limite)

  # Conversion des index de array -> liste
  # Sélection du Close à la date correspondant à l'index
  s_2s = df.loc[list(arr_e2s),'Close'].to_list()
  s_1m = df.loc[list(arr_e1m),'Close'].to_list()

  # Création d'un dataframe d'entrée selon les index indentifiés
  df_res = df.filter(items = entries_list, axis=0)

  # Ajout des séries comme nouvelles colonnes
  df_res['Close_2s'] = s_2s
  df_res['Close_1m'] = s_1m

  df_res['var_2s'] = round( (df_res['Close_2s']-df_res['Close'])/df_res['Close'] ,2)
  df_res['var_1m'] = round( (df_res['Close_1m']-df_res['Close'])/df_res['Close'] ,2)

  # Suppression de la dernière ligne, car n'ayant pas abouti dans le temps, va fausser les résultats d'analyse
  df_res.drop(df_res.tail(1).index, inplace = True)

  return df_res

In [ ]:
def get_exits_list(df, el, tp, sl):
  ''' Liste des sorties combinant risk management, si franchissement Supertrend Long, ou Take Profit / Stop Loss
  Paramètres : df_Analyse_Technique et liste des entrées '''
  
  exits_list=[]
  
  for e in range (0, len(el)):
    # On isole l'index de la ligne d'entrée
    row_index = el[e]
    # Pointeur commence les vérifications à partir de la ligne suivante
    r = row_index+1
    # Indicateur qu'une limite a été trouvée ou la fin des données atteinte
    touch = False

    # Définition des limites
    limite = df.loc[row_index,'Close'] * tp
    stop_loss = df.loc[row_index,'Close'] * sl
    
    # Test l'une des limites est atteinte à la cloture
    while(touch == False):  
      if( r >= df.shape[0]-1):
        exits_list.append(df.shape[0]-1)
        touch = True
      elif( 
            (df.loc[r,'Close'] >= limite) or 
            (df.loc[r,'Close'] <= stop_loss) or
            ( (df.loc[r,'Close'] < df.loc[r,'Supertrend']) and (df.loc[r,'Close_lag'] >= df.loc[r,'ST_lag']) )
           ):
        exits_list.append(r)
        touch = True
      else:
        r+=1
    e+=1

  return exits_list

In [ ]:
def generate_df_perf(df_indicateurs, le, ls):
  # Préparation au merge des entrées et sorties dans un seul et même DataFrame
  df_entries = df_indicateurs.filter(items = le, axis=0)
  df_entries = df_entries[['Date','Close']]
  df_entries.reset_index(drop=True, inplace=True)

  df_exits = df_indicateurs.filter(items = ls, axis=0)
  df_exits = df_exits[['Date','Close']]
  df_exits.reset_index(drop=True, inplace=True)

  # Jointure sur index réinitialisé
  df_P = df_entries.join(df_exits, lsuffix='_entry', rsuffix='_exit')
  df_P = df_P[['Date_entry','Date_exit','Close_entry','Close_exit']]
  # Variation positions Long
  df_P['var']= round( (df_P['Close_exit']-df_P['Close_entry'])/df_P['Close_entry'], 2)

  
  # Dédoublonnage itératif
  iteration = 1
  
  while iteration <4:
    
    doublons = []
    j=1
    # Obtention des dates de sorties, moins la dernière
    s_exit_dates = df_P["Date_exit"]
    s_exit_dates.drop(s_exit_dates.tail(1).index, inplace=True)
    
    # Obtention des index où les entrées chevauchent à une même date de sortie
    for date_sortie in s_exit_dates:
      if date_sortie > df_P["Date_entry"][j]:
        doublons.append(j)
      j+=1
    
    # Suppression des index identifiés comme doublons d'entrées
    df_P.drop(index=doublons, inplace=True)
    df_P.reset_index(drop=True, inplace=True)
    
    # Fin n-ième intération
    iteration+=1

  return df_P

In [ ]:
def calc_performance(df):
  ''' Calculer la performance cumulée en % '''

  # Calcul performance investissement initial
  perf_list=[1000]

  # Selection de la dernière colonne, variation à 1 mois
  perfs = df['var'].to_list()

  # Investissement de base, 1000
  invest = 1000

  # Calcul de la performance cumulée
  for i in range (0, df.shape[0]):
    invest = invest * (1 + perfs[i])
    perf_list.append(invest)

  # Equivalence en %
  perfP = round( ((perf_list[-1]-perf_list[0])/perf_list[0])*100 ,2)

  return perfP

In [ ]:
def calc_WinLossRatio(df):
  ''' Obtention du nombre de trades positifs à 2 semaines (10j)'''
  compte = df.apply(lambda x : 
                          True if x['var_2s'] > 0 
                          else False, axis = 1)
  # Count number of True in the series
  wr_ratio = round( len(compte[compte == True].index) / df.shape[0], 2)
  return wr_ratio

In [ ]:
def calc_profit_factors(df, WR):
  ''' Donne le profit factor classique et alternatif, sur la base du dataset df_perf et du Win Loss Ratio obtenu
  Profit factor classique = (Gross winning trades) / (Gross losing trades)
  Alternative = (Win Rate * Avg Win) / (Loss Rate * Avg Loss)'''
  pf=[]
  
  df_perf_pos = df.loc[ (df["var"] > 0) ]
  df_perf_pos["gain"] = 1000 * df_perf_pos["var"] 
  gross_gains = df_perf_pos["gain"].sum()

  df_perf_neg = df.loc[ (df["var"] <= 0) ]
  df_perf_neg["loss"] = 1000 * df_perf_neg["var"] 
  gross_losses = abs(df_perf_neg["loss"].sum())

  gross_profit_factor = round(gross_gains / gross_losses, 2)
  pf.append(gross_profit_factor)

  pfa_pos = df_perf_pos["var"].mean() * WR
  pfa_neg = ( abs(df_perf_neg["var"]).mean() ) * (1-WR)
  pfa = round(pfa_pos / pfa_neg, 2)
  pf.append(pfa)

  return pf

# Stratégie - Supertrend + VWMA #

In [ ]:
df_ta_SupertrendVWMA = generate_ta(df_source,13,3.5,10)

In [ ]:
df_ta_SupertrendVWMA.tail()

,Date,Close,Supertrend,VWMA
1317,2011-12-23,23.70,22.018488,22.91
1318,2011-12-27,23.91,22.018488,22.93
1319,2011-12-28,23.75,22.018488,23.01
1320,2011-12-29,23.84,22.018488,23.05
1321,2011-12-30,23.94,22.018488,23.32


In [ ]:
entries_list = find_entries(df_ta_SupertrendVWMA)

In [ ]:
df_entrees_perfs = generate_future_dates(df_ta_SupertrendVWMA,entries_list)

In [ ]:
w = calc_WinLossRatio(df_entrees_perfs)
print("Win rate à 10j :",w)

Win rate à 10j : 0.51


### Avec Money Management Exit ###

In [ ]:
liste_sorties = get_exits_list(df_ta_SupertrendVWMA, entries_list, 1.3, 0.9)
len(liste_sorties)

75

In [ ]:
# Préparation au merge des entrées et sorties dans un seul et même DataFrame
df_entries = df_ta_SupertrendVWMA.filter(items = entries_list, axis=0)
df_entries = df_entries[['Date','Close']]
df_entries.reset_index(drop=True, inplace=True)

df_exits = df_ta_SupertrendVWMA.filter(items = liste_sorties, axis=0)
df_exits = df_exits[['Date','Close']]
df_exits.reset_index(drop=True, inplace=True)

In [ ]:
# Jointure sur index réinitialisé
df_perf = df_entries.join(df_exits, lsuffix='_entry', rsuffix='_exit')
df_perf = df_perf[['Date_entry','Date_exit','Close_entry','Close_exit']]
# Variation positions Long
df_perf['var']= round( (df_perf['Close_exit']-df_perf['Close_entry'])/df_perf['Close_entry'], 2)
# Calcul de la durée entre entrée / sortie
df_perf['duree']= (df_perf['Date_exit']-df_perf['Date_entry'])/np.timedelta64(1,'D').astype(int)

Attention, doublons à appliquer 2x min

In [ ]:
# Obtention des dates de sorties, moins la dernière
s_exit_dates = df_perf["Date_exit"]
s_exit_dates.drop(s_exit_dates.tail(1).index, inplace=True)

# Obtention des index où plusieurs entrées correspondent à une même date de sortie
doublons = []
i=1
for date_sortie in s_exit_dates:
  if date_sortie > df_perf["Date_entry"][i]:
    doublons.append(i)
  i+=1

# Suppression des index identifiés comme doublons d'entrées
df_perf.drop(index=doublons, inplace=True)
df_perf.reset_index(drop=True, inplace=True)

In [ ]:
df_perf

,Date_entry,Date_exit,Close_entry,Close_exit,var,duree
0,2006-10-23,2006-11-28,19.19,19.16,-0.00,36 days
1,2007-01-11,2007-01-26,20.57,19.12,-0.07,15 days
2,2007-03-22,2007-07-27,18.89,21.58,0.14,127 days
3,2007-09-04,2007-11-08,22.27,22.90,0.03,65 days
4,2007-12-07,2008-01-03,22.49,20.63,-0.08,27 days
5,2008-04-02,2008-06-20,15.49,16.51,0.07,79 days
6,2008-08-07,2008-09-03,15.97,16.29,0.02,27 days
7,2009-02-06,2009-02-19,8.56,7.66,-0.11,13 days
8,2009-03-23,2009-04-30,7.70,10.26,0.33,38 days
9,2009-07-16,2009-10-21,10.76,14.00,0.30,97 days


In [ ]:
compte = df_perf.apply(lambda x : 
                        True if x['var'] > 0 
                        else False, axis = 1)
# Count number of True in the series
WR_ratio = round( len(compte[compte == True].index) / df_perf.shape[0], 2)
print("Win/Loss Ratio : ",WR_ratio)

Win/Loss Ratio :  0.52


In [ ]:
# Calcul performance investissement initial
perf_list=[1000]
#perfs = df_perf['var_norm']
perfs = df_perf['var']
invest = 1000
for i in range (0, df_perf.shape[0]):
  invest = invest*(1+perfs[i])
  perf_list.append(invest)
print("Buy & Hold 2006-2022 : {:.0f} %".format(bh1))
print("Performance cumulée : {} %".format(round( ((perf_list[-1]-perf_list[0])/perf_list[0])*100 ,2)) )

Buy & Hold 2006-2022 : 37 %
Performance cumulée : 128.44 %


<p>Résultats tests manuels, période 2006-2011</p>
<li>ST 10/3.5, MA 21, TpSL 1.3/0.9 -> 23 trades, WR 52%, Perf 59%</li>
<li>ST 10/3.5, MA 55, TpSL 1.3/0.9 -> 21 trades, WR 48%, Perf 53%</li>
<li>ST 10/3.5, MA 89, TpSL 1.3/0.9 -> 18 trades, WR 50%, Perf 83%<br />Peu de pertes dans perfs négatives, principalement trailing stop supertrend, fonctionne très bien aussi pour prises profits !</li>
<li>ST 10/3.5, MA 144, TpSL 1.3/0.9 -> 16 trades, WR 50%, Perf 29%<br />L'entrée semble trop tardive, 1 seule atteinte +30%</li>
<li>ST 10/3.5, MA 200, TpSL 1.3/0.9 -> 16 trades, WR 44%, Perf 14%</li>
<p>Meilleure combinaison : 13/3.5/10</p>
<li>Perf 128%, 21 Trades à 52% </li>
<li>Avec untop limite : Perf 83%, 21 trades à 52%. (max win +24%)</li>
<li>Si untop stop loss, Perf 112%, 20 trades à 55% (max loss -16%)</li>
<br/>
Année 2011 : <b>-19 %</b> pour une année "range"

### Supertrend + VWMA all Time ###

In [ ]:
df_ta_SupertrendVWMA = generate_ta(df_source_allTime,13,3.5,10)
entries_list = find_entries(df_ta_SupertrendVWMA)
df_entrees_perfs = generate_future_dates(df_ta_SupertrendVWMA,entries_list)

In [ ]:
w = calc_WinLossRatio(df_entrees_perfs)
print("Win rate à 10j, all time :",w)

Win rate à 10j, all time : 0.58


In [ ]:
liste_sorties = get_exits_list(df_ta_SupertrendVWMA, entries_list, 1.3, 0.9)
print("nombre de trades, avant filtrage : ",len(liste_sorties))

nombre de trades, avant filtrage :  260


In [ ]:
# Préparation au merge des entrées et sorties dans un seul et même DataFrame
df_entries = df_ta_SupertrendVWMA.filter(items = entries_list, axis=0)
df_entries = df_entries[['Date','Close']]
df_entries.reset_index(drop=True, inplace=True)

df_exits = df_ta_SupertrendVWMA.filter(items = liste_sorties, axis=0)
df_exits = df_exits[['Date','Close']]
df_exits.reset_index(drop=True, inplace=True)

In [ ]:
# Jointure sur index réinitialisé
df_perf = df_entries.join(df_exits, lsuffix='_entry', rsuffix='_exit')
df_perf = df_perf[['Date_entry','Date_exit','Close_entry','Close_exit']]
# Variation positions Long
df_perf['var']= round( (df_perf['Close_exit']-df_perf['Close_entry'])/df_perf['Close_entry'], 2)
# Calcul de la durée entre entrée / sortie
df_perf['duree']= (df_perf['Date_exit']-df_perf['Date_entry'])/np.timedelta64(1,'D').astype(int)

In [ ]:
# Obtention des dates de sorties, moins la dernière
s_exit_dates = df_perf["Date_exit"]
s_exit_dates.drop(s_exit_dates.tail(1).index, inplace=True)

# Obtention des index où plusieurs entrées correspondent à une même date de sortie
doublons = []
i=1
for date_sortie in s_exit_dates:
  if date_sortie > df_perf["Date_entry"][i]:
    doublons.append(i)
  i+=1

# Suppression des index identifiés comme doublons d'entrées
df_perf.drop(index=doublons, inplace=True)
df_perf.reset_index(drop=True, inplace=True)

In [ ]:
print("Nombre de trades, sans doublons, Long only : ",df_perf.shape[0])

Nombre de trades, sans doublons, Long only :  61


In [ ]:
df_perf

,Date_entry,Date_exit,Close_entry,Close_exit,var,duree
0,2006-10-23,2006-11-28,19.19,19.16,-0.00,36 days
1,2007-01-11,2007-01-26,20.57,19.12,-0.07,15 days
2,2007-03-22,2007-07-27,18.89,21.58,0.14,127 days
3,2007-09-04,2007-11-08,22.27,22.90,0.03,65 days
4,2007-12-07,2008-01-03,22.49,20.63,-0.08,27 days
...,...,...,...,...,...,...
56,2020-11-05,2021-02-23,510.00,576.40,0.13,110 days
57,2021-04-06,2021-05-11,637.20,591.20,-0.07,35 days
58,2021-06-10,2021-09-01,644.00,838.60,0.30,83 days
59,2021-10-19,2021-11-26,818.50,918.40,0.12,38 days


In [ ]:
compte = df_perf.apply(lambda x : 
                        True if x['var'] > 0 
                        else False, axis = 1)
# Count number of True in the series
WR_ratio = round( len(compte[compte == True].index) / df_perf.shape[0], 2)
print("Win/Loss Ratio all Time, long only : ",WR_ratio)

Win/Loss Ratio all Time, long only :  0.56


In [ ]:
# Calcul performance investissement initial
perf_list=[1000]
#perfs = df_perf['var_norm']
perfs = df_perf['var']
invest = 1000
for i in range (0, df_perf.shape[0]):
  invest = invest*(1+perfs[i])
  perf_list.append(invest)
print("Buy & Hold 2006-2022, Long only : {:.0f} %".format(bh2))
print("Performance cumulée, Long only : {:.0f} %".format(round( ((perf_list[-1]-perf_list[0])/perf_list[0])*100 ,2)) )

Buy & Hold 2006-2022, Long only : 4016 %
Performance cumulée, Long only : 1356 %


<p>Buy & Hold 2006-2022, Long only : 4016 %</p>
Meilleure combinaison 2006-2011 : 13/3.5/10<br />
Performance cumulée, Long only : 1356 %, 61 trades à 57 %<br />
<p> Meilleure combinaison all Time : 60/3.5/21<p>


## Comparaison avec données aléatoires ##

### Etat à +10j ###

In [ ]:
nb_trades = df_perf.shape[0]
index_max = df_ta_SupertrendVWMA.shape[0]
print("{} trades réalisés, index dates allant jusqu'à : {}".format(nb_trades, index_max))

18 trades réalisés, index dates allant jusqu'à : 1247


In [ ]:
i=0
random_WinRates = []

while i < 500:
  # Liste aléatoire dans la plage 0, index_max df, 
  #avec le nombre d'entrées correspondant au nombre d'entrées trades trouvées pour la stratégie à tester
  randomlist = random.sample(range(0, index_max-10), nb_trades)
  randomlist.sort()

  df_random_entries_perfs = generate_future_dates(df_ta_SupertrendVWMA, randomlist)

  random_WinRates.append( calc_WinLossRatio(df_random_entries_perfs) )
  i+=1

In [ ]:
print("WR median de {} données d'entrées aléatoires = {}".format(i, stats.median(random_WinRates)))

WR median de 500 données d'entrées aléatoires = 0.53


In [ ]:
fig = go.Figure()
fig.add_trace(go.Box(x=random_WinRates, name="Random Win Rate distribution 2006-2011", quartilemethod="inclusive"))

### Avec Risk Management ###

In [ ]:
i=0
random_WinRates = []
random_AggPerf = []

while i < 10000:
  # Liste aléatoire dans la plage 0, index_max df, 
  #avec le nombre d'entrées correspondant au nombre d'entrées trades trouvées pour la stratégie à tester
  # On prévoit une marge arbitraire en vue des doublons qui baisseront le nombres de trades au final
  randomlist = random.sample(range(0, index_max), nb_trades+6)
  randomlist.sort()

  liste_sorties = get_exits_list(df_ta_SupertrendVWMA, randomlist, 1.3, 0.9)

  # Préparation au merge des entrées et sorties dans un seul et même DataFrame
  df_entries = df_ta_SupertrendVWMA.filter(items = randomlist, axis=0)
  df_entries = df_entries[['Date','Close']]
  df_entries.reset_index(drop=True, inplace=True)

  df_exits = df_ta_SupertrendVWMA.filter(items = liste_sorties, axis=0)
  df_exits = df_exits[['Date','Close']]
  df_exits.reset_index(drop=True, inplace=True)

  # Jointure sur index réinitialisé
  df_perf = df_entries.join(df_exits, lsuffix='_entry', rsuffix='_exit')
  df_perf = df_perf[['Date_entry','Date_exit','Close_entry','Close_exit']]
  # Variation positions Long
  df_perf['var']= round( (df_perf['Close_exit']-df_perf['Close_entry'])/df_perf['Close_entry'], 2)

  # Obtention des dates de sorties, moins la dernière
  s_exit_dates = df_perf["Date_exit"]
  s_exit_dates.drop(s_exit_dates.tail(1).index, inplace=True)

  # Obtention des index où plusieurs entrées correspondent à une même date de sortie
  doublons = []
  j=1
  for date_sortie in s_exit_dates:
    if date_sortie > df_perf["Date_entry"][j]:
      doublons.append(j)
    j+=1

  # Suppression des index identifiés comme doublons d'entrées
  df_perf.drop(index=doublons, inplace=True)
  df_perf.reset_index(drop=True, inplace=True)

  # On ne va considérer la performance que si il y a strictement le même nombre de trades q
  # que dans notre stratégie à comparer avec les entrées aléatoires
  if df_perf.shape[0] == nb_trades:
    compte = df_perf.apply(lambda x : 
                            True if x['var'] > 0 
                            else False, axis = 1)
    # Count number of True in the series
    WR_ratio = round( len(compte[compte == True].index) / df_perf.shape[0], 2)
    random_WinRates.append( WR_ratio )

    # Calcul performance investissement initial
    perf_list=[1000]
    perfs = df_perf['var']
    invest = 1000
    for k in range (0, df_perf.shape[0]):
      invest = invest*(1+perfs[k])
      perf_list.append(invest)

    random_AggPerf.append( round( ((perf_list[-1]-perf_list[0])/perf_list[0])*100 ,2) )

  i+=1

In [ ]:
print("Mediane de {} données d'entrées aléatoires, avec Money Management = {}".format(len(random_AggPerf), stats.median(random_WinRates)))

Mediane de 1078 données d'entrées aléatoires, avec Money Management = 0.39


In [ ]:
fig = go.Figure()
fig.add_trace(go.Box(x=random_WinRates, name="Random Win Rate distribution 2006-2011, Money Management", quartilemethod="inclusive"))

In [ ]:
fig = go.Figure()
fig.add_trace(go.Box(x=random_AggPerf, name="Random Perf distribution 2006-2011, Money Management", quartilemethod="inclusive"))

## Supertrend multiples ##

In [ ]:
# Listes ATR et Multiplier pour Supertrend, et selection de VWMA #

np_list = np.arange(2.0,6.0,0.5)
l_atr = [x for x in range(5,120)]

d_vars_supertrend = {
    "atr": l_atr,
    "multiplier": list(np_list)
}

vwma_list = [5,10,21,34,55,70,89,144,200]

In [ ]:
dict_res = {
    "atr_length":[],
    "multiplier":[],
    "vwma_length":[],
    "cumulative_performance":[]
}
params_test=[]
  
for moy in vwma_list:
  for mult in d_vars_supertrend['multiplier']:
    for atr in d_vars_supertrend['atr']:

      # Dataframe contenant les indicateurs techniques
      #df_ta_ST_VWMA = generate_ta(df_source,atr,mult,moy)
      df_ta_ST_VWMA = generate_ta(df_source_allTime,atr,mult,moy)

      # Entrées & Sorties correspondantes à la stratégie
      entries_list = find_entries(df_ta_ST_VWMA)
      exits_list = get_exits_list(df_ta_ST_VWMA, entries_list, 1.3, 0.9)
      
      # DataFrame entrées - sorties - variations
      df_perf = generate_df_perf(df_ta_ST_VWMA, entries_list, exits_list)

      # Obtention de la performance cumulée
      perf = calc_performance(df_perf)

      # Inscription des résultats dans le dictionnaire
      dict_res["atr_length"].append(atr)
      dict_res["multiplier"].append(mult)
      dict_res["vwma_length"].append(moy)
      dict_res["cumulative_performance"].append(perf)

# Transformation en dataframe
df_multi_ST = pd.DataFrame.from_dict(dict_res)

In [ ]:
df_multi_ST.sort_values(by=["cumulative_performance"], ascending=False).head(20)

,atr_length,multiplier,vwma_length,cumulative_performance
2240,60,3.5,21,3008.06
2241,61,3.5,21,2960.96
2242,62,3.5,21,2933.14
2235,55,3.5,21,2905.59
2233,53,3.5,21,2861.98
2237,57,3.5,21,2790.41
2236,56,3.5,21,2790.41
2214,34,3.5,21,2777.08
2239,59,3.5,21,2768.34
2238,58,3.5,21,2768.34


In [ ]:
df_export = df_multi_ST.sort_values(by=["cumulative_performance"], ascending=False)
df_export.reset_index(drop=True, inplace=True)
#df_export.to_excel("/content/drive/MyDrive/Colab Notebooks/Perf_SuperTrend_VWMA.xlsx", index=False, header=True)
df_export.to_excel("/content/drive/MyDrive/Colab Notebooks/Perf_SuperTrend_VWMA_allTime.xlsx", index=False, header=True)

In [ ]:
fig = px.scatter_3d(df_multi_ST, x='vwma_length', y='atr_length', z='multiplier',
              color='cumulative_performance', title="Performances période 2006-2011. Supertrend + VWMA")
fig.show()

In [ ]:
df_multi_ST.sort_values(by=["cumulative_performance"], ascending=False).head(20)

,atr_length,multiplier,vwma_length,cumulative_performance
2240,60,3.5,21,3008.06
2241,61,3.5,21,2960.96
2242,62,3.5,21,2933.14
2235,55,3.5,21,2905.59
2233,53,3.5,21,2861.98
2237,57,3.5,21,2790.41
2236,56,3.5,21,2790.41
2214,34,3.5,21,2777.08
2239,59,3.5,21,2768.34
2238,58,3.5,21,2768.34


In [ ]:
fig = px.scatter_3d(df_multi_ST, x='vwma_length', y='atr_length', z='multiplier',
              color='cumulative_performance', title="Performances période 2006-2022. Supertrend + VWMA")
fig.show()

Performance très intéressante pour vwma length 21<br/>
2 combinaisons de Supertrend :
<li> atr 34, multiplier 3.5 (meilleur en All Time également)</li>
<li> atr 17, multiplier 4</li>

In [ ]:
fig = px.scatter_3d(df_multi_ST, x='multiplier', y='atr_length', z='cumulative_performance',
              color='vwma_length')
fig.show()

Autre vue des clusters à environ 35 et 17.<br/>
Le croisement à VWMA 21 surperforme de très loin les valeurs plus hautes (et 8 semble lui trop court).<br/>
Les multiplier aussi sont très nettement localisés à 3.5 et 4.

# Stratégie Supertrend X MA #

## Fonctions dédiées ##

In [ ]:
def generate_ta_ST_MA(df_in,atr,multip,ma_l):
  ''' Genère un Dataframe contenant Supertrend + VWMA '''

  df = df_in.ta.supertrend(high=df_in["High"], low=df_in["Low"], close=df_in["Close"], length = atr, multiplier = multip)

  # Nom des colonnes pour saisir la 1ere variabilisée
  col_list = df.columns
  # Suppresion des lignes vides, Subset sur la colonne supertrend
  df.dropna(subset=[col_list[0]], inplace=True)
  # Suppression de la 1ère ligne dans la valeur supertrend est = 0
  s_supertrend = df.iloc[1:,0]

  df_st = df_in.join(s_supertrend)

  # Suppression des lignes sans correspondance entre date et une valeur supertrend
  col_list = df_st.columns
  # Colonne supertrend étant la dernière
  supertrend_col = col_list[-1]
  # Garder uniquement date, close et supertrend
  df_st = df_st[["Close",supertrend_col]]
  # Suppression des lignes sans indicateur
  df_st.dropna(subset=[supertrend_col], inplace=True)
  # Arrondi colonne supertrend
  df_st[supertrend_col] = round( df_st[supertrend_col] ,2)

  # Ajout colonne MA
  df_st["ma_"+str(ma_l)] = round( df_in.ta.sma(close=df_in["Close"], length=ma_l) ,2)

  # Idem : suppression des lignes sans correspondace entre date et vwma
  df_st.dropna(subset=["ma_"+str(ma_l)], inplace=True)
  
  # Retour des dates en colonne plutôt qu'index
  df_st.reset_index(inplace=True)

  df_st.columns=['Date','Close','Supertrend','MA']
  
  return df_st

In [ ]:
def find_entries_ST_MA_long(df_in):
  ''' Obtenir les index des dates auxquelles un croisement vers le haut est effectué par Supertrend sur MA
  et que le prix est > ST '''

  df = df_in.copy()
  # Ajout des colonnes lag
  shift_ma = df[df.columns[-1]].shift(1)
  shift_st = df[df.columns[-2]].shift(1)

  shift_ma[0]=shift_ma[1]
  df['MA_lag']=shift_ma

  shift_st[0]=shift_st[1]
  df['ST_lag']=shift_st

  e_l = df.index[ 
                  ( (df["Supertrend"] > df["MA"]) 
                  & (df["ST_lag"] <= df["MA_lag"]) 
                  & (df["Close"] >= df["Supertrend"]) )
                ].to_list()
  
  e_s = df.index[ 
                  ( (df["Supertrend"] < df["MA"]) 
                  & (df["ST_lag"] >= df["MA_lag"]) 
                  & (df["Close"] <= df["Supertrend"]) )
                ].to_list()

  return e_l, e_s

In [ ]:
def generate_future_dates_Short(df, entries_list):
  '''Générer les dates futures après identification des entrées
  Puis calcul des variations'''

  arr_entries = np.asarray(entries_list)

  arr_e2s = arr_entries + 10
  arr_e1m = arr_entries + 21
  
  # Aggregation des matrices sous forme de liste en vue de traitements en boucle
  dates_ech = [arr_e2s, arr_e1m]

  # On fixe le plus grand index possible, au cas où la projection 5, 10j ou 1 mois en avant soit out of bounds
  date_limite = df.shape[0]-1

  # Correction des éventuelles dates futures out of bounds, selon fonction définie précedemment
  for echeance in dates_ech:
    echeance = fix_oob_index(echeance,date_limite)

  # Conversion des index de array -> liste
  # Sélection du Close à la date correspondant à l'index
  s_2s = df.loc[list(arr_e2s),'Close'].to_list()
  s_1m = df.loc[list(arr_e1m),'Close'].to_list()

  # Création d'un dataframe d'entrée selon les index indentifiés
  df_res = df.filter(items = entries_list, axis=0)

  # Ajout des séries comme nouvelles colonnes
  df_res['Close_2s'] = s_2s
  df_res['Close_1m'] = s_1m

  # Variation inverse prenant en compte la position Short
  df_res['var_2s'] = round( (df_res['Close']-df_res['Close_2s'])/df_res['Close'] ,2)
  df_res['var_1m'] = round( (df_res['Close']-df_res['Close_1m'])/df_res['Close'] ,2)

  # Suppression de la dernière ligne, car n'ayant pas abouti dans le temps, va fausser les résultats d'analyse
  df_res.drop(df_res.tail(1).index, inplace = True)

  return df_res

In [ ]:
def check_21d_interval(entrees):
  i=0
  while i < len(entrees)-1 :
    if entrees[i+1] - entrees[i] <=21:
      entrees.pop(i+1)
    i+=1

In [ ]:
def get_exits_list_Long_ST_MA(df_in, el, tp, sl):
  ''' Liste des sorties combinant risk management, si franchissement Supertrend Long, ou Take Profit / Stop Loss
  Paramètres : df_Analyse_Technique et liste des entrées '''
  
  df = df_in.copy()
  # Ajout des colonnes lag
  shift_st = df[df.columns[-2]].shift(1)
  shift_close = df["Close"].shift(1)

  shift_close[0]=shift_close[1]
  df['Close_lag']=shift_close

  shift_st[0]=shift_st[1]
  df['ST_lag']=shift_st


  exits_list=[]
  
  for e in range (0, len(el)):
    # On isole l'index de la ligne d'entrée
    row_index = el[e]
    # Pointeur commence les vérifications à partir de la ligne suivante
    r = row_index+1
    # Indicateur qu'une limite a été trouvée ou la fin des données atteinte
    touch = False

    # Définition des limites
    limite = df.loc[row_index,'Close'] * tp
    stop_loss = df.loc[row_index,'Close'] * sl
    
    # Test l'une des limites est atteinte à la cloture
    while touch == False :  
      if r >= df.shape[0]-1 :
        exits_list.append(df.shape[0]-1)
        touch = True
      elif( 
            (df.loc[r,'Close'] >= limite) or 
            (df.loc[r,'Close'] <= stop_loss) or
            ( (df.loc[r,'Close'] < df.loc[r,'Supertrend']) and (df.loc[r,'Close_lag'] >= df.loc[r,'ST_lag']) )
           ):
        exits_list.append(r)
        touch = True
      else:
        r+=1
    e+=1

  return exits_list

In [ ]:
def get_exits_list_Short_ST_MA(df_in, el, tp, sl):
  ''' Liste des sorties combinant risk management, si franchissement Supertrend Long, ou Take Profit / Stop Loss
  Paramètres : df_Analyse_Technique et liste des entrées '''
  
  df = df_in.copy()
  # Ajout des colonnes lag
  shift_st = df[df.columns[-2]].shift(1)
  shift_close = df["Close"].shift(1)

  shift_close[0]=shift_close[1]
  df['Close_lag']=shift_close

  shift_st[0]=shift_st[1]
  df['ST_lag']=shift_st


  exits_list=[]
  
  for e in range (0, len(el)):
    # On isole l'index de la ligne d'entrée
    row_index = el[e]
    # Pointeur commence les vérifications à partir de la ligne suivante
    r = row_index+1
    # Indicateur qu'une limite a été trouvée ou la fin des données atteinte
    touch = False

    # Définition des limites
    limite = df.loc[row_index,'Close'] * tp
    stop_loss = df.loc[row_index,'Close'] * sl
    
    # Test l'une des limites est atteinte à la cloture
    while touch == False :  
      if r >= df.shape[0]-1 :
        exits_list.append(df.shape[0]-1)
        touch = True
      elif( 
            (df.loc[r,'Close'] >= limite) or 
            (df.loc[r,'Close'] <= stop_loss) or
            ( (df.loc[r,'Close'] < df.loc[r,'Supertrend']) and (df.loc[r,'Close_lag'] >= df.loc[r,'ST_lag']) )
           ):
        exits_list.append(r)
        touch = True
      else:
        r+=1
    e+=1

  return exits_list

## Analyse Stat des entrées ##

### Période 2006-2011 ###

In [ ]:
df_ta_ST_MA = generate_ta_ST_MA(df_source,34,3,70)

In [ ]:
df_ta_ST_MA.head(1)

,Date,Close,Supertrend,MA
0,2007-01-10,19.55,19.82,19.07


In [ ]:
# necessaire avant la fonction generate future dates, index numérique et non dates
df_source.reset_index(inplace=True)
df_source_allTime.reset_index(inplace=True)

In [ ]:
long_entries, short_entries = find_entries_ST_MA_long(df_ta_ST_MA)
nb_trades_long = len(long_entries)
nb_trades_short = len(short_entries)
print("Nombre de trades long brut : {}\nNombre de trades short brut : {}".format(nb_trades_long,nb_trades_short))

Nombre de trades long brut : 14
Nombre de trades short brut : 12


In [ ]:
check_21d_interval(long_entries)
check_21d_interval(short_entries)
print("Nombre de trades long : {}\nNombre de trades short : {}".format(len(long_entries),len(short_entries)))

Nombre de trades long : 13
Nombre de trades short : 12


In [ ]:
# df pour stats perf à +10j et +21j
df_stats_long = generate_future_dates(df_source, long_entries)
df_stats_short = generate_future_dates_Short(df_source, short_entries)

KeyError: ignored

In [ ]:
fig = go.Figure()
# Use x instead of y argument for horizontal plot
fig.add_trace(go.Box(x=df_stats_short['var_2s'], name="STxMA Short standard sortie +2s", quartilemethod="inclusive", marker_color='indianred'))
fig.add_trace(go.Box(x=df_stats_long['var_2s'], name="STxMA Long standard sortie +2s", quartilemethod="inclusive", marker_color = 'lightseagreen'))
fig.add_trace(go.Box(x=df_stats_short['var_1m'], name="STxMA Short standard sortie +1m", quartilemethod="inclusive", marker_color='indianred'))
fig.add_trace(go.Box(x=df_stats_long['var_1m'], name="STxMA Long standard sortie +1m", quartilemethod="inclusive", marker_color = 'lightseagreen'))
fig.update(layout_showlegend=False)
fig.show()

NameError: ignored

In [ ]:
df_stats_long

,Date,Open,High,Low,Close,Volume,Close_2s,Close_1m,var_2s,var_1m
74,2007-01-17,20.53,20.58,20.40,20.53,966,19.10,19.64,-0.07,-0.04
179,2007-06-19,21.53,21.56,21.40,21.46,1465,21.51,23.19,0.00,0.08
330,2008-01-23,15.65,15.88,14.87,14.87,28727,15.82,15.46,0.06,0.04
403,2008-05-08,17.14,17.24,16.93,17.13,19971,16.76,17.25,-0.02,0.01
565,2008-12-30,7.01,7.01,6.88,6.88,40001,7.19,7.77,0.05,0.13
633,2009-04-08,8.36,8.65,8.36,8.56,452,9.56,9.43,0.12,0.10
715,2009-08-06,12.19,12.26,12.08,12.11,1068,12.20,12.36,0.01,0.02
739,2009-09-09,12.56,12.92,12.56,12.92,6704,13.70,13.55,0.06,0.05
792,2009-11-23,14.00,14.46,14.00,14.34,11365,14.40,15.70,0.00,0.09
841,2010-02-02,14.72,14.87,14.25,14.80,74849,15.58,16.67,0.05,0.13


### Période 2006 - aujourd'hui ###

In [ ]:
df_ta_ST_MA = generate_ta_ST_MA(df_source_allTime,34,3,70)

long_entries, short_entries = find_entries_ST_MA_long(df_ta_ST_MA)

check_21d_interval(long_entries)
check_21d_interval(short_entries)
print("Nombre de trades long : {}\nNombre de trades short : {}".format(len(long_entries),len(short_entries)))

Nombre de trades long : 39
Nombre de trades short : 26


In [ ]:
# df pour stats perf à +10j et +21j
df_stats_long = generate_future_dates(df_source_allTime, long_entries)
df_stats_short = generate_future_dates_Short(df_source_allTime, short_entries)

In [ ]:
fig = go.Figure()
# Use x instead of y argument for horizontal plot
fig.add_trace(go.Box(x=df_stats_short['var_2s'], name="STxMA Short standard sortie +2s", quartilemethod="inclusive", marker_color='indianred'))
fig.add_trace(go.Box(x=df_stats_long['var_2s'], name="STxMA Long standard sortie +2s", quartilemethod="inclusive", marker_color = 'lightseagreen'))
fig.add_trace(go.Box(x=df_stats_short['var_1m'], name="STxMA Short standard sortie +1m", quartilemethod="inclusive", marker_color='indianred'))
fig.add_trace(go.Box(x=df_stats_long['var_1m'], name="STxMA Long standard sortie +1m", quartilemethod="inclusive", marker_color = 'lightseagreen'))
fig.update(layout_showlegend=False)
fig.show()

#### Combinaison Long & Short, avec paramètres identiques ####

In [ ]:
'''Etude de la performance en combinant Long & Short
NB : Les paramètres optimaux ne sont peut-être pas les mêmes !'''
df_stats_long["position"] = "Long"
df_stats_short["position"] = "Short"


df_stats_LS = pd.concat([df_stats_long,df_stats_short])
df_stats_LS.sort_values(by=["Date"], inplace=True)

df_stats_LS.reset_index(drop=True, inplace=True)

Le dédoublonnage des positions Long / Short élimine pour 3/4 des positions Short retournées en Long<br />Confirmation de la brièveté des impulsions à la baisse

In [ ]:
 # Dédoublonnage itératif, adapté à sortie fixe + 21j
iteration = 1

while iteration <4:
  
  doublons = []
  j=1
  # Obtention des dates de sorties, moins la dernière
  s_exit_dates = df_stats_LS["Date"]
  s_exit_dates.drop(s_exit_dates.tail(1).index, inplace=True)
  
  # Obtention des index où les entrées chevauchent à une même date de sortie
  for date_sortie in s_exit_dates:
    if  (df_stats_LS["Date"][j] - date_sortie).days < 21:
      doublons.append(j)
    j+=1
  
  # Suppression des index identifiés comme doublons d'entrées
  df_stats_LS.drop(index=doublons, inplace=True)
  df_stats_LS.reset_index(drop=True, inplace=True)
  
  # Fin n-ième intération
  iteration+=1

In [ ]:
fig = go.Figure()
# Use x instead of y argument for horizontal plot
fig.add_trace(go.Box(x=df_stats_LS['var_2s'], name="STxMA L&S standard sortie +2s", quartilemethod="inclusive", marker_color='darkblue'))
fig.add_trace(go.Box(x=df_stats_LS['var_1m'], name="STxMA L&S standard sortie +1m", quartilemethod="inclusive", marker_color='darkblue'))
fig.update(layout_showlegend=False)
fig.show()

## Recherche de la meilleure combinaison paramètre ST et longueur MA ##

In [ ]:
# Listes ATR et Multiplier pour Supertrend, et plage de longueur MA #

np_list = np.arange(2.0,6.0,0.5)
l_atr = [x for x in range(5,80)]

d_vars_supertrend = {
    "atr": l_atr,
    "multiplier": list(np_list)
}

ma_list = [e for e in range(5,201,5)]

In [ ]:
dict_res = {
    "atr_length":[],
    "multiplier":[],
    "ma_length":[],
    "cumulative_performance":[]
}
params_test=[]
  
for moy in ma_list:
  for mult in d_vars_supertrend['multiplier']: 
    for atr in d_vars_supertrend['atr']:

      # Dataframe contenant les indicateurs techniques
      #df_ta_ST_MA = generate_ta_ST_MA(df_source,atr,mult,moy)
      df_ta_ST_MA = generate_ta_ST_MA(df_source_allTime,atr,mult,moy)

      # Entrées & Sorties correspondantes à la stratégie
      long_entries, short_entries = find_entries_ST_MA_long(df_ta_ST_MA)
      
      # Focus uniquement sur positions longues
      exits_list = get_exits_list_Long_ST_MA(df_ta_ST_MA, long_entries, 1.3, 0.9)
      
      # DataFrame entrées - sorties - variations
      df_perf = generate_df_perf(df_ta_ST_MA, long_entries, exits_list)

      # Obtention de la performance cumulée
      perf = calc_performance(df_perf)

      # Inscription des résultats dans le dictionnaire
      dict_res["atr_length"].append(atr)
      dict_res["multiplier"].append(mult)
      dict_res["ma_length"].append(moy)
      dict_res["cumulative_performance"].append(perf)

# Transformation en dataframe
df_multi_ST = pd.DataFrame.from_dict(dict_res)

In [ ]:
df_multi_ST.to_excel("/content/drive/MyDrive/Colab Notebooks/Perf_SuperTrend_MA_allTime.xlsx", index=False, header=True)

# Stratégie triple Supertrend #

## Triple Supertrend fixe #
10x1, 11x2, 12x3

### Génération Indicateurs ###

In [ ]:
def generate_ta_ST(df_hist,df_in,atr_list,multip_list):
  ''' Genère un Dataframe contenant Supertrend 
  Répeter pour ajout d'autant de colonnes correspondantes'''
  
  for l in range (0, len(atr_list)):
    
    df = df_hist.ta.supertrend(high=df_hist["High"], low=df_hist["Low"], close=df_hist["Close"], length = atr_list[l], multiplier = multip_list[l])

    # On ne conserve que la première colonne
    df = df[[df.columns[0]]]

    # Suppression de la 1ère ligne inutile
    df = df.iloc[1: , :] 

    # Ajout de la colonne Supertrend, jointure sur index Dates
    df_in = df_in.join(df)

  # Suppression de toutes les lignes vides
  df_in.dropna(inplace=True) 
  
  return df_in

In [ ]:
# Initiation de DataFrame TA prêt à recevoir les 3 Supertrend
df_3ST = df_source_allTime[["Close"]].copy()
df_3ST.head(1)

,Close
Date,
2006-10-02,17.44


In [ ]:
l_atr = [10,11,12]
l_multip = [1,2,3]

In [ ]:
df_3ST = generate_ta_ST(df_source_allTime, df_3ST, l_atr, l_multip)

In [ ]:
df_3ST.shape

(3937, 4)

In [ ]:
df_3ST

,Date,Close,SUPERT_10_1.0,SUPERT_11_2.0,SUPERT_12_3.0,Uptrend,Downtrend,Uptrend_exit,Downtrend_exit
0,2006-10-18,18.68,19.142662,18.178435,17.656164,False,False,False,False
1,2006-10-19,18.55,18.905125,18.178435,17.656164,False,False,False,False
2,2006-10-20,18.55,18.905125,18.178435,17.656164,False,False,False,False
3,2006-10-23,19.19,18.561926,18.178435,17.795642,True,False,False,False
4,2006-10-24,18.90,18.601510,18.225269,17.850522,True,False,False,False
...,...,...,...,...,...,...,...,...,...
3932,2022-03-21,727.60,694.161879,660.479683,626.524205,True,False,False,False
3933,2022-03-22,755.10,713.945691,680.076984,645.988855,True,False,False,False
3934,2022-03-23,759.20,721.756122,689.024531,655.902284,True,False,False,False
3935,2022-03-24,755.90,721.756122,689.024531,656.477093,True,False,False,False


### Identification des entrées ###



In [ ]:
col_list = df_3ST.columns

In [ ]:
df_3ST["Uptrend"]=df_3ST.apply(lambda x: 
                                True if x['Close'] > x[col_list[1]]
                                  and x[col_list[1]] > x[col_list[2]]
                                  and x[col_list[2]] > x[col_list[3]]
                                else False,axis=1)

In [ ]:
# Même principe à la recherche de Shorts
# Attention 1 colonne en plus donc décaler les index de colonnes de -1 par rapport à la requête précédente
df_3ST["Downtrend"]=df_3ST.apply(lambda x: 
                                 True if x['Close'] < x[col_list[1]]
                                  and x[col_list[1]] < x[col_list[2]]
                                  and x[col_list[2]] < x[col_list[3]]
                                else False,axis=1)

In [ ]:
df_3ST["Uptrend_exit"]=df_3ST.apply(lambda x: 
                                    True if x['Close'] < x[col_list[1]]
                                    and x[col_list[1]] < x[col_list[2]]
                                    and x['Close'] > x[col_list[3]]
                                else False,axis=1)

In [ ]:
df_3ST["Downtrend_exit"]=df_3ST.apply(lambda x: 
                                      True if x['Close'] > x[col_list[1]]
                                      and x[col_list[1]] > x[col_list[2]]
                                      and x['Close'] < x[col_list[3]]
                                else False,axis=1)

In [ ]:
df_3ST

,Date,Close,SUPERT_10_1.0,SUPERT_11_2.0,SUPERT_12_3.0,Uptrend,Downtrend,Uptrend_exit,Downtrend_exit
0,2006-10-18,18.68,19.142662,18.178435,17.656164,False,False,False,False
1,2006-10-19,18.55,18.905125,18.178435,17.656164,False,False,False,False
2,2006-10-20,18.55,18.905125,18.178435,17.656164,False,False,False,False
3,2006-10-23,19.19,18.561926,18.178435,17.795642,True,False,False,False
4,2006-10-24,18.90,18.601510,18.225269,17.850522,True,False,False,False
...,...,...,...,...,...,...,...,...,...
3932,2022-03-21,727.60,694.161879,660.479683,626.524205,True,False,False,False
3933,2022-03-22,755.10,713.945691,680.076984,645.988855,True,False,False,False
3934,2022-03-23,759.20,721.756122,689.024531,655.902284,True,False,False,False
3935,2022-03-24,755.90,721.756122,689.024531,656.477093,True,False,False,False


In [ ]:
df_3ST.reset_index(inplace=True)

In [ ]:
long_entry = []
for row in range(1, df_3ST.shape[0]):
  if df_3ST.at[row,"Uptrend"] == True and df_3ST.at[row-1,"Uptrend"] == False:
    long_entry.append(row)

In [ ]:
len(long_entry)

200

In [ ]:
short_entry = []
for row in range(1, df_3ST.shape[0]):
  if df_3ST.at[row,"Downtrend"] == True and df_3ST.at[row-1,"Downtrend"] == False:
    short_entry.append(row)

In [ ]:
len(short_entry)

152

### Sortie à date fixe ###

#### Analyse entrées Long ####

In [ ]:
# Cette fonction nécessite d'avoir les numéros en index, plus les dates
df_analyse_entrees_long = generate_future_dates(df_3ST, long_entry)

In [ ]:
# Pour attraper les 3 colonnes Supertrend
ST_col = df_analyse_entrees_long.columns
# On conserve uniquement les colonnes pertinentes avec Long
df_analyse_entrees_long = df_analyse_entrees_long[["Date","Close","Close_2s","var_2s","Close_1m","var_1m",ST_col[2],ST_col[3],ST_col[4]]]
df_analyse_entrees_long.tail()

,Date,Close,Close_2s,var_2s,Close_1m,var_1m,SUPERT_10_1.0,SUPERT_11_2.0,SUPERT_12_3.0
3752,2021-07-12,751.1,782.3,0.04,780.1,0.04,736.505771,725.518584,712.342343
3760,2021-07-22,761.8,784.4,0.03,784.6,0.03,742.577867,727.240745,721.177916
3769,2021-08-04,772.8,774.2,0.00,833.9,0.08,753.548256,749.668798,734.752059
3822,2021-10-18,802.9,873.4,0.09,942.5,0.17,773.398893,751.967724,730.870255
3843,2021-11-16,942.5,926.9,-0.02,886.2,-0.06,914.418271,897.080472,877.768179


In [ ]:
wr = calc_WinLossRatio(df_analyse_entrees_long)
print("Win Rate Long entries : {} %\nPour {} trades".format(wr, len(long_entry)))

Win Rate Long entries : 0.54 %
Pour 200 trades


#### Analyse entrées Short ####

In [ ]:
# Cette fonction nécessite d'avoir les numéros en index, plus les dates
df_analyse_entrees_short = generate_future_dates(df_3ST, short_entry)

# Nouveau calcul des variations en considérant les positions Short, donc bénéfice inversé
df_analyse_entrees_short.drop(columns=["var_2s","var_1m"], inplace=True)
df_analyse_entrees_short["var_2s"] = round( (df_analyse_entrees_short["Close_2s"] - df_analyse_entrees_short["Close"])/df_analyse_entrees_short["Close"] ,2)
df_analyse_entrees_short["var_1m"] = round( (df_analyse_entrees_short["Close_1m"] - df_analyse_entrees_short["Close"])/df_analyse_entrees_short["Close"] ,2)

# Reorganisation des colonnes
df_analyse_entrees_short = df_analyse_entrees_short[["Date","Close","Close_2s","var_2s","Close_1m","var_1m",ST_col[2],ST_col[3],ST_col[4]]]
df_analyse_entrees_short.tail()

,Date,Close,Close_2s,var_2s,Close_1m,var_1m,SUPERT_10_1.0,SUPERT_11_2.0,SUPERT_12_3.0
3851,2021-11-26,918.4,928.3,0.01,962.1,0.05,954.867837,979.518559,1003.334549
3867,2021-12-20,852.9,953.9,0.12,820.1,-0.04,896.498558,932.243842,966.805366
3880,2022-01-06,881.4,822.2,-0.07,735.3,-0.17,907.525374,938.690078,966.805366
3886,2022-01-14,839.2,719.2,-0.14,719.0,-0.14,868.585849,903.568573,937.037501
3907,2022-02-14,719.0,714.4,-0.01,637.4,-0.11,741.102848,777.298961,813.630396


In [ ]:
wr = calc_WinLossRatio(df_analyse_entrees_short)
print("Win Rate Long entries : {} %\nPour {} trades".format(wr, len(short_entry)))

Win Rate Long entries : 0.57 %
Pour 152 trades


In [ ]:
fig = go.Figure()
# Use x instead of y argument for horizontal plot
fig.add_trace(go.Box(x=df_analyse_entrees_short['var_2s'], name="3ST Short standard sortie +1s", quartilemethod="inclusive", marker_color='indianred'))
fig.add_trace(go.Box(x=df_analyse_entrees_long['var_2s'], name="3ST Long standard sortie +1s", quartilemethod="inclusive", marker_color = 'lightseagreen'))
fig.add_trace(go.Box(x=df_analyse_entrees_short['var_1m'], name="3ST Short standard sortie +1m", quartilemethod="inclusive", marker_color='indianred'))
fig.add_trace(go.Box(x=df_analyse_entrees_long['var_1m'], name="3ST Long standard sortie +1m", quartilemethod="inclusive", marker_color = 'lightseagreen'))
fig.update(layout_showlegend=False)
fig.show()

### Sorties avec money management ###

In [ ]:
# Compte tenu des WR < 60%, on applique +30/-10
# Cf Risk management optimisé sur entrées aléatoires, spécifique Lyx NDQ2x
take_profit_factor = 1.3
stop_loss_factor = 0.9

#### Fonctions adaptées à 3ST ####

In [ ]:
def get_exits_list_long(df, el, tp, sl):
  ''' Liste des sorties combinant risk management, si franchissement Supertrend Long, ou Take Profit / Stop Loss
  Paramètres : df_Analyse_Technique et liste des entrées '''
  
  exits_list=[]
  
  for e in range (0, len(el)):
    # On isole l'index de la ligne d'entrée
    row_index = el[e]
    # Pointeur commence les vérifications à partir de la ligne suivante
    r = row_index+1
    # Indicateur qu'une limite a été trouvée ou la fin des données atteinte
    touch = False

    # Définition des limites
    limite = df.loc[row_index,'Close'] * tp
    stop_loss = df.loc[row_index,'Close'] * sl
    
    # Test l'une des limites est atteinte à la cloture
    while(touch == False):  
      if( r >= df.shape[0]-1):
        exits_list.append(df.shape[0]-1)
        touch = True
      elif( 
            (df.loc[r,'Close'] >= limite) or 
            (df.loc[r,'Close'] <= stop_loss) or
            (df.loc[r,'Uptrend_exit'] == True ) or
           # En cas de double cassure des 2 dernières Supertrend Support
            ( df.loc[r,'Close'] < df.loc[r,df_3ST.columns[-6]] and  df.loc[r,'Close'] <= df.loc[r,df_3ST.columns[-5]] )
           ):
        exits_list.append(r)
        touch = True
      else:
        r+=1
    e+=1

  return exits_list

In [ ]:
def get_exits_list_short(df, el, tp, sl):
  ''' Liste des sorties combinant risk management, si franchissement Supertrend Long, ou Take Profit / Stop Loss
  Paramètres : df_Analyse_Technique et liste des entrées '''
  
  exits_list=[]
  
  for e in range (0, len(el)):
    # On isole l'index de la ligne d'entrée
    row_index = el[e]
    # Pointeur commence les vérifications à partir de la ligne suivante
    r = row_index+1
    # Indicateur qu'une limite a été trouvée ou la fin des données atteinte
    touch = False

    # Définition des limites
    # Adaptation à l'invserse avec les short
    limite = df.loc[row_index,'Close'] * (1-(tp-1))
    stop_loss = df.loc[row_index,'Close'] * (1+(1-sl))
    
    # Test l'une des limites est atteinte à la cloture
    while(touch == False):  
      if( r >= df.shape[0]-1):
        exits_list.append(df.shape[0]-1)
        touch = True
      # Inversion des signes, positions Short
      elif( 
            (df.loc[r,'Close'] <= limite) or 
            (df.loc[r,'Close'] >= stop_loss) or
            (df.loc[r,'Downtrend_exit'] == True ) or
            ( df.loc[r,'Close'] > df.loc[r,df_3ST.columns[-6]] and  df.loc[r,'Close'] >= df.loc[r,df_3ST.columns[-5]] )
           ):
        exits_list.append(r)
        touch = True
      else:
        r+=1
    e+=1

  return exits_list

In [ ]:
def generate_df_perf_short(df_indicateurs, le, ls):
  # Préparation au merge des entrées et sorties dans un seul et même DataFrame
  df_entries = df_indicateurs.filter(items = le, axis=0)
  df_entries = df_entries[['Date','Close']]
  df_entries.reset_index(drop=True, inplace=True)

  df_exits = df_indicateurs.filter(items = ls, axis=0)
  df_exits = df_exits[['Date','Close']]
  df_exits.reset_index(drop=True, inplace=True)

  # Jointure sur index réinitialisé
  df_P = df_entries.join(df_exits, lsuffix='_entry', rsuffix='_exit')
  df_P = df_P[['Date_entry','Date_exit','Close_entry','Close_exit']]
  # Variation positions Long
  df_P['var']= round( (df_P['Close_entry']-df_P['Close_exit'])/df_P['Close_entry'], 2)

  # Obtention des dates de sorties, moins la dernière
  s_exit_dates = df_P["Date_exit"]
  s_exit_dates.drop(s_exit_dates.tail(1).index, inplace=True)

  # Obtention des index où plusieurs entrées correspondent à une même date de sortie
  doublons = []
  j=1
  for date_sortie in s_exit_dates:
    if date_sortie > df_P["Date_entry"][j]:
      doublons.append(j)
    j+=1

  # Suppression des index identifiés comme doublons d'entrées
  df_P.drop(index=doublons, inplace=True)
  df_P.reset_index(drop=True, inplace=True)

  return df_P

In [ ]:
def calc_WinRate(df):
  ''' Obtention du nombre de trades positifs à 2 semaines (10j)'''
  compte = df.apply(lambda x : 
                          True if x['var'] > 0 
                          else False, axis = 1)
  # Count number of True in the series
  wr_ratio = round( len(compte[compte == True].index) / df.shape[0], 2)
  return wr_ratio

#### Application ####

In [ ]:
long_exits = get_exits_list_long(df_3ST, long_entry, take_profit_factor, stop_loss_factor)
short_exits = get_exits_list_short(df_3ST, short_entry, take_profit_factor, stop_loss_factor)

In [ ]:
if len(short_exits) == len(short_entry) and len(long_exits) == len(long_entry):
  print('ok')
else:
  print('mon vier')

ok


In [ ]:
df_perf_long = generate_df_perf(df_3ST, long_entry, long_exits)
df_perf_long.name = "df_perf_long"

df_perf_short = generate_df_perf_short(df_3ST, short_entry, short_exits)
df_perf_short.name = "df_perf_short"

In [ ]:
df_perf_long

,Date_entry,Date_exit,Close_entry,Close_exit,var
0,2006-10-23,2006-11-02,19.19,18.32,-0.05
1,2006-11-07,2006-11-27,19.14,19.40,0.01
2,2007-01-11,2007-01-18,20.57,19.87,-0.03
3,2007-02-22,2007-02-27,20.08,19.18,-0.04
4,2007-03-22,2007-05-16,18.89,19.67,0.04
...,...,...,...,...,...
118,2021-04-01,2021-05-04,610.10,609.90,-0.00
119,2021-06-04,2021-07-19,629.00,727.00,0.16
120,2021-07-22,2021-09-13,761.80,810.40,0.06
121,2021-10-18,2021-11-23,802.90,939.00,0.17


In [ ]:
len(df_perf_long[df_perf_long["var"]>0].index) / df_perf_long.shape[0]

0.4634146341463415

In [ ]:
df_perf_long.shape[0]

123

In [ ]:
df_perf_short.shape[0]

109

In [ ]:
df_perf_short

,Date_entry,Date_exit,Close_entry,Close_exit,var
0,2006-11-27,2006-12-15,19.40,19.92,-0.03
1,2006-12-19,2007-01-10,18.95,19.55,-0.03
2,2007-01-25,2007-02-07,19.55,19.78,-0.01
3,2007-02-13,2007-02-20,19.13,19.85,-0.04
4,2007-02-27,2007-03-08,19.18,18.34,0.04
...,...,...,...,...,...
104,2021-11-26,2021-12-07,918.40,947.00,-0.03
105,2021-12-20,2021-12-23,852.90,941.00,-0.10
106,2022-01-06,2022-02-02,881.40,789.40,0.10
107,2022-02-14,2022-02-25,719.00,697.60,0.03


In [ ]:
# Combinaison des dataframes Long & Short
df_perf_double = pd.concat([df_perf_long, df_perf_short])
df_perf_double.sort_values(by=["Date_entry"], inplace=True)
df_perf_double.name = "df_perf_double"

In [ ]:
fig = go.Figure()
# Use x instead of y argument for horizontal plot
fig.add_trace(go.Box(x=df_perf_double['var'], name="3ST Double Strategy", quartilemethod="inclusive", marker_color='darkblue'))
fig.add_trace(go.Box(x=df_perf_short['var'], name="3ST Short Strategy", quartilemethod="inclusive", marker_color='indianred'))
fig.add_trace(go.Box(x=df_perf_long['var'], name="3ST Long Strategy", quartilemethod="inclusive", marker_color = 'lightseagreen'))
fig.update(layout_showlegend=False)
fig.show()

In [ ]:
df_dict = {
    "name": ["df_perf_double", "df_perf_long", "df_perf_short"],
    "data": [df_perf_double, df_perf_long, df_perf_short]
}

In [ ]:
for d in range (0,len(df_dict["name"])):
  wr = calc_WinRate(df_dict["data"][d])
  print( "Win Rate {} : {} %, pour {} trades".format(df_dict["name"][d], wr, df_dict["data"][d].shape[0]) )

Win Rate df_perf_double : 0.36 %, pour 232 trades
Win Rate df_perf_long : 0.46 %, pour 123 trades
Win Rate df_perf_short : 0.24 %, pour 109 trades


In [ ]:
for d in range (0,len(df_dict["name"])):
  p = calc_performance(df_dict["data"][d])
  print( "Performance {} : {} %".format(df_dict["name"][d], p ) )

Performance df_perf_double : -58.88 %
Performance df_perf_long : 279.2 %
Performance df_perf_short : -89.16 %


# Aides #

In [ ]:
help(ta.supertrend)

Help on function supertrend in module pandas_ta.overlap.supertrend:

supertrend(high, low, close, length=None, multiplier=None, offset=None, **kwargs)
    Supertrend (supertrend)
    
    Supertrend is an overlap indicator. It is used to help identify trend
    direction, setting stop loss, identify support and resistance, and/or
    generate buy & sell signals.
    
    Sources:
        http://www.freebsensetips.com/blog/detail/7/What-is-supertrend-indicator-its-calculation
    
    Calculation:
        Default Inputs:
            length=7, multiplier=3.0
        Default Direction:
            Set to +1 or bullish trend at start
    
        MID = multiplier * ATR
        LOWERBAND = HL2 - MID
        UPPERBAND = HL2 + MID
    
        if UPPERBAND[i] < FINAL_UPPERBAND[i-1] and close[i-1] > FINAL_UPPERBAND[i-1]:
            FINAL_UPPERBAND[i] = UPPERBAND[i]
        else:
            FINAL_UPPERBAND[i] = FINAL_UPPERBAND[i-1])
    
        if LOWERBAND[i] > FINAL_LOWERBAND[i-1] and clos

In [ ]:
help(ta.vwma)

Help on function vwma in module pandas_ta.overlap.vwma:

vwma(close, volume, length=None, offset=None, **kwargs)
    Volume Weighted Moving Average (VWMA)
    
    Volume Weighted Moving Average.
    
    Sources:
        https://www.motivewave.com/studies/volume_weighted_moving_average.htm
    
    Calculation:
        Default Inputs:
            length=10
        SMA = Simple Moving Average
        pv = close * volume
        VWMA = SMA(pv, length) / SMA(volume, length)
    
    Args:
        close (pd.Series): Series of 'close's
        volume (pd.Series): Series of 'volume's
        length (int): It's period. Default: 10
        offset (int): How many periods to offset the result. Default: 0
    
    Kwargs:
        fillna (value, optional): pd.DataFrame.fillna(value)
        fill_method (value, optional): Type of fill method
    
    Returns:
        pd.Series: New feature generated.



In [ ]:
help(ta.sma)

Help on function sma in module pandas_ta.overlap.sma:

sma(close, length=None, talib=None, offset=None, **kwargs)
    Simple Moving Average (SMA)
    
    The Simple Moving Average is the classic moving average that is the equally
    weighted average over n periods.
    
    Sources:
        https://www.tradingtechnologies.com/help/x-study/technical-indicator-definitions/simple-moving-average-sma/
    
    Calculation:
        Default Inputs:
            length=10
        SMA = SUM(close, length) / length
    
    Args:
        close (pd.Series): Series of 'close's
        length (int): It's period. Default: 10
        talib (bool): If TA Lib is installed and talib is True, Returns the TA Lib
            version. Default: True
        offset (int): How many periods to offset the result. Default: 0
    
    Kwargs:
        adjust (bool): Default: True
        presma (bool, optional): If True, uses SMA for initial value.
        fillna (value, optional): pd.DataFrame.fillna(value)
     

In [ ]:
help(ta.ichimoku)

Help on function ichimoku in module pandas_ta.overlap.ichimoku:

ichimoku(high, low, close, tenkan=None, kijun=None, senkou=None, include_chikou=True, offset=None, **kwargs)
    Ichimoku Kinkō Hyō (ichimoku)
    
    Developed Pre WWII as a forecasting model for financial markets.
    
    Sources:
        https://www.tradingtechnologies.com/help/x-study/technical-indicator-definitions/ichimoku-ich/
    
    Calculation:
        Default Inputs:
            tenkan=9, kijun=26, senkou=52
        MIDPRICE = Midprice
        TENKAN_SEN = MIDPRICE(high, low, close, length=tenkan)
        KIJUN_SEN = MIDPRICE(high, low, close, length=kijun)
        CHIKOU_SPAN = close.shift(-kijun)
    
        SPAN_A = 0.5 * (TENKAN_SEN + KIJUN_SEN)
        SPAN_A = SPAN_A.shift(kijun)
    
        SPAN_B = MIDPRICE(high, low, close, length=senkou)
        SPAN_B = SPAN_B.shift(kijun)
    
    Args:
        high (pd.Series): Series of 'high's
        low (pd.Series): Series of 'low's
        close (pd.Serie

# Zone de test #

## Indicateurs techniques ##

In [ ]:
# supertrend(high, low, close, length=None, multiplier=None, offset=None, **kwargs)
df_supertrend = df_source.ta.supertrend(high=df_source["High"], low=df_source["Low"], close=df_source["Close"], length=34, multiplier=3.5)

In [ ]:
df_supertrend

,SUPERT_34_3.5,SUPERTd_34_3.5,SUPERTl_34_3.5,SUPERTs_34_3.5
Date,,,,
2006-10-02,0.000000,1,NaN,NaN
2006-11-17,18.763917,1,18.763917,NaN
2006-11-20,18.765536,1,18.765536,NaN
2006-11-21,19.084268,1,19.084268,NaN
2006-11-22,19.084268,1,19.084268,NaN
...,...,...,...,...
2011-12-23,21.822842,1,21.822842,NaN
2011-12-27,21.822842,1,21.822842,NaN
2011-12-28,21.822842,1,21.822842,NaN


In [ ]:
# Nom des colonnes pour saisir la 1ere variabilisée
col_list = df_supertrend.columns
# Subset sur la 1ere colonne
df_supertrend.dropna(subset=[col_list[0]], inplace=True)
#df_supertrend = df_supertrend[[[col_list[0]]]]

In [ ]:
s_supertrend = df_supertrend.iloc[1:,0]

In [ ]:
s_supertrend

Date
2006-11-17    18.763917
2006-11-20    18.765536
2006-11-21    19.084268
2006-11-22    19.084268
2006-11-23    19.169835
                ...    
2011-12-23    21.822842
2011-12-27    21.822842
2011-12-28    21.822842
2011-12-29    21.822842
2011-12-30    21.822842
Name: SUPERT_34_3.5, Length: 1301, dtype: float64

In [ ]:
df_ta = df_source.join(s_supertrend)

In [ ]:
df_ta

,Open,High,Low,Close,Volume,SUPERT_34_3.5
Date,,,,,,
2006-10-02,17.63,17.63,17.44,17.44,590,NaN
2006-10-03,16.90,16.98,16.88,16.98,61,NaN
2006-10-04,16.98,17.59,16.98,17.59,2,NaN
2006-10-05,18.00,18.11,17.96,18.10,996,NaN
2006-10-06,18.17,18.30,18.17,18.30,1302,NaN
...,...,...,...,...,...,...
2011-12-23,23.61,23.70,23.48,23.70,17797,21.822842
2011-12-27,24.00,24.02,23.69,23.91,8386,21.822842
2011-12-28,23.81,23.97,23.75,23.75,43541,21.822842


In [ ]:
col_list = df_ta.columns
# Colonne supertrend étant la dernière
supertrend_col = col_list[-1]
# Garder uniquement date, close et supertrend
df_ta = df_ta[["Close",supertrend_col]]
# Suppression des lignes sans indicateur
df_ta.dropna(subset=[supertrend_col], inplace=True)
df_ta

,Close,SUPERT_34_3.5
Date,,
2006-11-17,19.99,18.763917
2006-11-20,20.17,18.765536
2006-11-21,20.30,19.084268
2006-11-22,20.27,19.084268
2006-11-23,20.35,19.169835
...,...,...
2011-12-23,23.70,21.822842
2011-12-27,23.91,21.822842
2011-12-28,23.75,21.822842


In [ ]:
# Ajout colonne VWM
vwma_length = 21
df_ta["vwma_"+str(vwma_length)] = round( df_source.ta.vwma(close=df_source["Close"], volume=df_source["Volume"], length=vwma_length) ,2)

In [ ]:
df_ta.dropna(subset=["vwma_"+str(vwma_length)], inplace=True)
df_ta

,Close,SUPERT_34_3.5,vwma_21
Date,,,
2006-11-17,19.99,18.763917,19.41
2006-11-20,20.17,18.765536,19.46
2006-11-21,20.30,19.084268,19.47
2006-11-22,20.27,19.084268,19.48
2006-11-23,20.35,19.169835,19.49
...,...,...,...
2011-12-23,23.70,21.822842,23.23
2011-12-27,23.91,21.822842,23.25
2011-12-28,23.75,21.822842,23.33


In [ ]:
# vwma(close, volume, length=None, offset=None, **kwargs)

l_med = 21
l_long = 55
df_source["vwma_"+str(l_med)] = round( df_source.ta.vwma(close=df_source["Close"], volume=df_source["Volume"], length=l_med) ,2)
df_source["vwma_"+str(l_long)] = round( df_source.ta.vwma(close=df_source["Close"], volume=df_source["Volume"], length=l_long) ,2)

## Entrées ##

In [ ]:
# Ajout des colonnes lag
shift_vwma = df_ta[df_ta.columns[-1]].shift(1)
shift_st = df_ta[df_ta.columns[-2]].shift(1)
shift_close = df_ta["Close"].shift(1)

shift_vwma[0]=shift_vwma[1]
df_ta['VWMA_lag']=shift_vwma

shift_st[0]=shift_st[1]
df_ta['ST_lag']=shift_st

shift_close[0]=shift_close[1]
df_ta['Close_lag']=shift_close

In [ ]:
df_ta.head()

,Close,SUPERT_34_3.5,vwma_21,VWMA_lag,ST_lag,Close_lag
Date,,,,,,
2006-11-17,19.99,18.763917,19.41,19.41,18.763917,19.99
2006-11-20,20.17,18.765536,19.46,19.41,18.763917,19.99
2006-11-21,20.30,19.084268,19.47,19.46,18.765536,20.17
2006-11-22,20.27,19.084268,19.48,19.47,19.084268,20.30
2006-11-23,20.35,19.169835,19.49,19.48,19.084268,20.27


In [ ]:
entries_list = df_ta.index[ 
                             ( (df_ta["Close"] > df_ta["SUPERT_34_3.5"]) 
                             & (df_ta["Close_lag"] <= df_ta["ST_lag"]) 
                             & (df_ta["Close"] >= df_ta["vwma_21"]) )
                             |  
                             ( (df_ta["Close"] > df_ta["vwma_21"]) 
                             & (df_ta["Close_lag"] <= df_ta["VWMA_lag"]) 
                             & (df_ta["Close"] >= df_ta["SUPERT_34_3.5"]) )
                             ].to_list()

In [ ]:
print("Nombre de trades : ",len(entries_list))

Nombre de trades :  59


In [ ]:
df_entrees_perfs = generate_future_dates(df_ta,entries_list)

TypeError: ignored

## Verif Perf ##

In [ ]:
df_verif = generate_ta(df_source_allTime,34,3.5,21)
el = find_entries(df_verif)
df_j10 = generate_future_dates(df_verif,el)

In [ ]:
w = calc_WinLossRatio(df_j10)
print("Win rate à 10j :",w)

Win rate à 10j : 0.58


In [ ]:
exl = get_exits_list(df_verif, el, 1.3, 0.9)

In [ ]:
len(exl) == len(el)

True

In [ ]:
df_perf_verif = generate_df_perf(df_verif, el, exl)

In [ ]:
print("Nombre de trades : ",df_perf_verif.shape[0])

Nombre de trades :  62


In [ ]:
compte = df_perf_verif.apply(lambda x : 
                        True if x['var'] > 0 
                        else False, axis = 1)
# Count number of True in the series
WR_ratio = round( len(compte[compte == True].index) / df_perf_verif.shape[0], 2)
print("Win/Loss Ratio : ",WR_ratio)

Win/Loss Ratio :  0.55


In [ ]:
# Calcul performance investissement initial
perf_list=[1000]
perfs = df_perf_verif['var']
invest = 1000
for i in range (0, df_perf_verif.shape[0]):
  invest = invest*(1+perfs[i])
  perf_list.append(invest)
round( ((perf_list[-1]-perf_list[0])/perf_list[0])*100 ,2)

3907.01

In [ ]:
bh_AT = round(((df_source_allTime['Close'].iloc[-1] - df_source_allTime['Close'].iloc[0])/df_source_allTime['Close'].iloc[0])*100,2)
print("Performance Buy & Hold all time : ",bh_AT)

Performance Buy & Hold all time :  4271.56


In [ ]:
df_perf_verif

,Date_entry,Date_exit,Close_entry,Close_exit,var
0,2007-01-11,2007-01-26,20.57,19.12,-0.07
1,2007-03-22,2007-06-08,18.89,20.04,0.06
2,2007-06-15,2007-07-27,21.59,21.58,-0.00
3,2007-08-30,2007-11-08,21.26,22.90,0.08
4,2007-12-07,2008-01-03,22.49,20.63,-0.08
...,...,...,...,...,...
57,2021-03-16,2021-05-11,598.20,591.20,-0.01
58,2021-06-09,2021-08-30,635.80,830.80,0.31
59,2021-10-18,2021-11-26,802.90,918.40,0.14
60,2021-12-27,2022-01-07,962.10,857.60,-0.11


In [ ]:
# Vérification du bon déroulé de la fonction
profit_factors = calc_profit_factors(df_perf_verif, WR_ratio)
print(profit_factors[0])
print(profit_factors[1])

4.03
4.06
